In [1]:
import numpy as np

import torch
import torch.nn as nn

from torch_geometric.nn import GCNConv

In [2]:
class GCN(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, 2)
        self.conv2 = GCNConv(2, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x_copy = x.detach().numpy().transpose()
        print("x after first layer")
        print(x_copy, x_copy.shape)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        return x

In [3]:
class GCNConvByHand(nn.Module):
    def __init__(self, dim_in, dim_out):
        super().__init__()
        self.linear = torch.nn.Linear(dim_in, dim_out, bias=True)

    def forward(self, x, A):
        num_nodes = A.shape[0]
        omega_k = self.linear.weight
        beta_k = self.linear.bias.reshape(-1, 1)
        # print("omega_k", omega_k, "beta_k", beta_k)
        H_k = x
        x = torch.matmul(
            beta_k, torch.reshape(torch.ones(num_nodes), (1, -1))
        ) + torch.matmul(omega_k, torch.matmul(H_k, A + torch.eye(num_nodes)))
        return x

In [4]:
class GCNByHand(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConvByHand(in_channels, in_channels)
        self.conv2 = GCNConvByHand(in_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        print("x after first layer\n", x)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        return x

In [5]:
num_nodes = 3
num_features = 2

x = torch.randn((num_features, num_nodes))
# print("x", x, x.shape)

edge_index = torch.tensor([[0, 0, 1, 2], [1, 2, 0, 0]], dtype=torch.int64)

A = torch.tensor([[0, 1, 1], [1, 0, 0], [1, 0, 0]])  # adjacency matrix

model = GCNByHand(in_channels=num_features, out_channels=num_features)

out = model(x, A)

print("output")
print(out)

x after first layer
 tensor([[ 0.1753, -0.0294,  0.0372],
        [ 0.2103, -0.2469,  0.0838]], grad_fn=<AddBackward0>)
output
tensor([[0.1357, 0.1501, 0.1357],
        [0.2755, 0.3207, 0.2755]], grad_fn=<AddBackward0>)


In [6]:
A = np.array(A)

H_k = np.array(x)
# H_k

# H_k__A = H_k @ A
# H_k__A, H_k__A.shape

omega_0 = np.array(model.conv1.linear.weight.detach().numpy())
beta_0 = np.array(model.conv1.linear.bias.detach().numpy()).reshape((-1, 1))
# print(omega_0, beta_0)


preactivation = beta_0 @ np.ones(num_nodes).reshape((1, -1)) + omega_0 @ H_k @ (
    A + np.identity(num_nodes)
)
preactivation

array([[ 0.17526821, -0.02942616,  0.03722825],
       [ 0.210296  , -0.24688188,  0.08381679]])